In [ ]:
#  Our Configuration
# import the necessary packages
import torch
import os

# base path of the dataset
DATASET_PATH = "your path/tiled_images"

# define the path to the images and masks dataset
IMAGE_DATASET_PATH = os.path.join(DATASET_PATH, "images")
MASK_DATASET_PATH = os.path.join(DATASET_PATH, "labels")

# define the test split
TEST_SPLIT = 0.2

# determine the device to be used for training and evaluation
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# determine if we will be pinning memory during data loading
PIN_MEMORY = True if DEVICE == "cuda" else False

MODEL = ["deeplabv3_resnet50", "fcn_resnet101"]
# initialize learning rate, number of epochs to train for, and the
# batch size
INIT_LR = 0.01
NUM_EPOCHS = 20
BATCH_SIZE = 26
# define the input image dimensions
INPUT_IMAGE_WIDTH = 224
INPUT_IMAGE_HEIGHT = 224
# define threshold to filter weak predictions
THRESHOLD = 0.5
# define the path to the base output directory
BASE_OUTPUT = "your path/output"
# define the path to the output serialized model, model training
# plot, and testing image paths
MODEL_PATH = os.path.join(BASE_OUTPUT, "hts_deeplab.pth")
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot_hts_deeplab.png"])
TEST_PATHS = os.path.sep.join([BASE_OUTPUT, "test_paths.txt"])

In [ ]:
# Custom Segmentation Dataset

# import the necessary packages
from torch.utils.data import Dataset
import cv2

class SegmentationDataset(Dataset):
	def __init__(self, imagePaths, maskPaths, transforms):
		# store the image and mask filepaths, and augmentation
		# transforms
		self.imagePaths = imagePaths
		self.maskPaths = maskPaths
		self.transforms = transforms

	def __len__(self):
		# return the number of total samples contained in the dataset
		return len(self.imagePaths)

	def __getitem__(self, idx):
		# grab the image path from the current index
		imagePath = self.imagePaths[idx]

		# load the image from disk, swap its channels from BGR to RGB,
		# and read the associated mask from disk in grayscale mode
		image = cv2.imread(imagePath)
		image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
		mask = cv2.imread(self.maskPaths[idx], 0)

		# check to see if we are applying any transformations
		if self.transforms is not None:
			# apply the transformations to both image and its mask
			image = self.transforms(image)
			mask =( self.transforms(mask))*255

		# return a tuple of the image and its mask
		return (image, mask)

In [ ]:
from torch.nn import BCEWithLogitsLoss
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import time
import os

In [ ]:
# load the image and mask filepaths in a sorted manner
imagePaths = sorted(list(paths.list_images(IMAGE_DATASET_PATH)))
maskPaths = sorted(list(paths.list_images(MASK_DATASET_PATH)))
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
split = train_test_split(imagePaths, maskPaths,
	test_size=TEST_SPLIT, random_state=42)
# unpack the data split
(trainImages, testImages) = split[:2]
(trainMasks, testMasks) = split[2:]
# write the testing image paths to disk so that we can use then
# when evaluating/testing our model
print("[INFO] saving testing image paths...")
f = open(TEST_PATHS, "w")
f.write("\n".join(testImages))
f.close()

In [ ]:
# define transformations
transforms = transforms.Compose([transforms.ToPILImage(),
 	transforms.Resize((INPUT_IMAGE_HEIGHT,
		INPUT_IMAGE_WIDTH)),
	transforms.ToTensor()])
# create the train and test datasets
trainDS = SegmentationDataset(imagePaths=trainImages, maskPaths=trainMasks,
	transforms=transforms)
testDS = SegmentationDataset(imagePaths=testImages, maskPaths=testMasks,
    transforms=transforms)
print(f"[INFO] found {len(trainDS)} examples in the training set...")
print(f"[INFO] found {len(testDS)} examples in the test set...")
# create the training and test data loaders
trainLoader = DataLoader(trainDS, shuffle=True,
	batch_size=BATCH_SIZE, pin_memory=PIN_MEMORY,
	num_workers=os.cpu_count())
testLoader = DataLoader(testDS, shuffle=False,
	batch_size=BATCH_SIZE, pin_memory=PIN_MEMORY,
	num_workers=os.cpu_count())



In [ ]:
model = torch.hub.load("pytorch/vision:v0.10.0", MODEL[0],
	pretrained=False)
model.to(DEVICE)
lossFunc = BCEWithLogitsLoss()
opt = Adam(model.parameters(), lr=INIT_LR)
# calculate steps per epoch for training and test set
trainSteps = len(trainDS) // BATCH_SIZE
testSteps = len(testDS) // BATCH_SIZE
# initialize a dictionary to store training history
H = {"train_loss": [], "test_loss": []}

In [ ]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(model)


In [ ]:
# loop over epochs
print("[INFO] training the network...")
startTime = time.time()
for e in tqdm(range(NUM_EPOCHS)):
	# set the model in training mode
	model.train()
	# initialize the total training and validation loss
	totalTrainLoss = 0
	totalTestLoss = 0
	# loop over the training set
	for (i, (x, y)) in enumerate(trainLoader):
		# send the input to the device
		(x, y) = (x.to(DEVICE), y.to(DEVICE))
		# perform a forward pass and calculate the training loss
		pred = model(x)["out"]
		pred2 =pred[:,1:2,:,:]
		loss = lossFunc(pred2, y)
		# first, zero out any previously accumulated gradients, then
		# perform backpropagation, and then update model parameters
		opt.zero_grad()
		loss.backward()
		opt.step()
		# add the loss to the total training loss so far
		totalTrainLoss += loss
	# switch off autograd
	with torch.no_grad():
		# set the model in evaluation mode
		model.eval()
		# loop over the validation set
		for (x, y) in testLoader:
			# send the input to the device
			(x, y) = (x.to(DEVICE), y.to(DEVICE))
			# make the predictions and calculate the validation loss
			pred = model(x)["out"]
			pred2 =pred[:,1:2,:,:]
			totalTestLoss += lossFunc(pred2, y)
	# calculate the average training and validation loss
	avgTrainLoss = totalTrainLoss / trainSteps
	avgTestLoss = totalTestLoss / testSteps
	# update our training history
	H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
	H["test_loss"].append(avgTestLoss.cpu().detach().numpy())
	# print the model training and validation information
	print("[INFO] EPOCH: {}/{}".format(e + 1, NUM_EPOCHS))
	print("Train loss: {:.6f}, Test loss: {:.4f}".format(
		avgTrainLoss, avgTestLoss))
# display the total time needed to perform the training
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(
	endTime - startTime))

In [ ]:
# plot the training loss
plt.figure()
plt.plot(H["train_loss"], label="train_loss")
plt.plot(H["test_loss"], label="test_loss")
plt.title("Training Loss on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig(PLOT_PATH)
# serialize the model to disk
torch.save(model, MODEL_PATH)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import cv2
import os
from sklearn.metrics import accuracy_score
from sklearn.metrics import jaccard_score

import pandas as pd

def prepare_plot(origImage, origMask, predMask):
	# initialize our figure
	figure, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 10))
	# plot the original image, its mask, and the predicted mask
	ax[0].imshow(origImage)
	ax[1].imshow(origMask)
	ax[2].imshow(predMask)
	# set the titles of the subplots
	ax[0].set_title("Image")
	ax[1].set_title("Original Mask")
	ax[2].set_title("Predicted Mask")

	# set the layout of the figure and display it
	figure.tight_layout()
	figure.show()

In [ ]:
!pip install monai

In [ ]:
import numpy as np
from monai.metrics.utils import get_mask_edges, get_surface_distance

def hausdorff(y_true, y_pred):
    h, w = 224,224
    max_dist = np.sqrt(w ** 2 + h ** 2)
    (edges_pred, edges_gt) = get_mask_edges(y_pred, y_true)
    surface_distance = get_surface_distance(edges_pred, edges_gt,
                                            distance_metric="euclidean")
    if surface_distance.shape == (0,):
        return 1.0
    dist = np.max(surface_distance)
    if dist > max_dist:
        return 0.0
    return dist / max_dist

In [ ]:
def metri(true_masks, predicted_masks):
  iou = jaccard_score(true_masks, predicted_masks, average='weighted')
  dice_coefficient = 2*iou / (1 + iou)
  tp=(true_masks*predicted_masks).sum()
  tn=(np.logical_not(true_masks)*np.logical_not(predicted_masks)).sum()
  fn=((true_masks)*np.logical_not(predicted_masks)).sum()
  fp=(np.logical_not(true_masks)*(predicted_masks)).sum()
  recall=tp/(tp+fn)
  precison=tp/(tp+fp)
  accuracy=(tp+tn)/(tp+tn+fp+fn)
  hd95=hausdorff(true_masks,predicted_masks)

  return accuracy,iou,dice_coefficient,recall,precison,hd95

In [ ]:
def make_predictions(model, imagePath):
	# set model to evaluation mode
	model.eval()
	# turn off gradient tracking
	with torch.no_grad():
		# load the image from disk, swap its color channels, cast it
		# to float data type, and scale its pixel values
		image = cv2.imread(imagePath)
		image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
		image = image.astype("float32") / 255.0
		# resize the image and make a copy of it for visualization
		image = cv2.resize(image, (224, 224))
		orig = image.copy()
		# find the filename and generate the path to ground truth
		# mask
		filename = imagePath.split(os.path.sep)[-1]
		groundTruthPath = os.path.join(MASK_DATASET_PATH,
			filename.replace('bmp', 'tif'))
		# load the ground-truth segmentation mask in grayscale mode
		# and resize it
		gtMask = cv2.imread(groundTruthPath, 0)

		gtMask = cv2.resize(gtMask, (INPUT_IMAGE_HEIGHT,INPUT_IMAGE_HEIGHT))

		# make the channel axis to be the leading one, add a batch
		# dimension, create a PyTorch tensor, and flash it to the
		# current device
		image = np.transpose(image, (2, 0, 1))
		image = np.expand_dims(image, 0)
		image = torch.from_numpy(image).to(DEVICE)
		# make the prediction, pass the results through the sigmoid
		# function, and convert the result to a NumPy array
		predMask = model(image)["out"]
		predMask = predMask[:,1:2,:,:]
		predMask = torch.sigmoid(predMask)
		predMask = predMask.cpu().numpy()
		# filter out the weak predictions and convert them to integers
		predMask = (predMask > THRESHOLD)
		predMask = predMask.astype(np.uint8)
		# prepare a plot for visualization

		prepare_plot(orig, gtMask, predMask[0,0,:,:])
		return gtMask, predMask[0,0,:,:]
# load the image paths in our testing file and randomly select 10
# image paths
print("[INFO] loading up test image paths...")
imagePaths = open(TEST_PATHS).read().strip().split("\n")
# load our model from disk and flash it to the current device
print("[INFO] load up model...")
model = torch.hub.load("pytorch/vision:v0.10.0", MODEL[0],
	pretrained=False)
model = torch.load(MODEL_PATH).to(DEVICE)
acc=[]
iou=[]
dice=[]
recall=[]
prec=[]
HD95=[]
# iterate over the randomly selected test image paths
for path in imagePaths:
	print(path)
	myMask,myPred=make_predictions(model, path)
	acc1,iou1,dice1,recall1,prec1,hausdorffdis1=metri(myMask,myPred)
	print(acc1)
	print(iou1)
	print(dice1)
	print(recall1)
	print(prec1)
	print(hausdorffdis1)

	acc.append(acc1)
	iou.append(iou1)
	dice.append(dice1)
	recall.append(recall1)
	prec.append(prec1)
	HD95.append(hausdorffdis1)
	# make predictions and visualize the results
print(f"acc: {np.mean(acc)} var ={np.var(acc)}")
print(f"jaccard: {np.mean(iou) } var ={np.var(iou)}")
print(f"Dice: {np.mean(dice)} var ={np.var(dice)}")
print(f"Recall: {np.mean(recall)} var ={np.var(recall)}")
print(f"Precison: {np.mean(prec)} var ={np.var(prec)}")
print(f"Hausdorff distance: {np.mean(HD95)} var ={np.var(HD95)}")